# Disclaimer:

DeepSphere.AI developed these materials based on its team’s expertise and technical infrastructure, and we are sharing these materials strictly for learning and research.
These learning resources may not work on other learning infrastructures and DeepSphere.AI advises the learners to use these materials at their own risk. As needed, we will
be changing these materials without any notification and we have full ownership and accountability to make any change to these materials.

<pre><b>Author :                          Chief Architect :       Reviewer :</b></pre>
<hr>
<pre>Avinash R & Jothi Periasamy       Jothi Periasamy         Jothi Periasamy</pre>

# Import Libraries and Install Dependencies

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras.optimizers import SGD
import random
import warnings

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense

warnings.filterwarnings('ignore')

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

# Class ChatBot
File Descriptions :
- DSAI_intents.json - input data as json file
- DSAI_words.pkl - lemmatized words stored as pickle format
- DSAI_classes.pkl - classes are stored as pickle format
- DSAI_chatbot_model.h5 - saved trained model

In [2]:
class ChatBot:

  def __init__(self, intents): #constructor
  #intents - data from json file
    self.vAR_words=[]
    self.vAR_classes = []
    self.vAR_documents = []
    self.vAR_ignore_words = ['?', '!']
    self.vAR_intents = intents
    self.vAR_lemmatizer = WordNetLemmatizer()
    
    for intent in self.vAR_intents['intents']:
      for pattern in intent['patterns']:

        vAR_w = nltk.word_tokenize(pattern) # take each word and tokenize it
        self.vAR_words.extend(vAR_w)
        self.vAR_documents.append((vAR_w, intent['tag']))  # documents = (list of tokenized words in patterns, tag)

        if intent['tag'] not in self.vAR_classes: # adding classes to our class list
            self.vAR_classes.append(intent['tag'])

    self.vAR_words = [self.vAR_lemmatizer.lemmatize(w.lower()) for w in self.vAR_words if w not in self.vAR_ignore_words]
    self.vAR_words = sorted(list(set(self.vAR_words)))
    self.vAR_classes = sorted(list(set(self.vAR_classes)))
    
    #Examples
    #documents - [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['What', "'s", 'up'], 'greeting')]
    #classes ['Contact', 'Textbook', 'accomplishments', 'ask']
    #words - lemmatized words ["'", "'s", 'a', 'accomplishment', 'achievement', 'any']

    print(len(self.vAR_documents), "documents", self.vAR_documents, end='\n\n')
    print(len(self.vAR_classes), "classes", self.vAR_classes, end='\n\n')
    print(len(self.vAR_words), "unique lemmatized words", self.vAR_words, end='\n\n')

  def store_object(self): 
    #store/dump lemmatized words and classes as pickle file
    pickle.dump(self.vAR_words,open('../Utility/DSAI_Tbot_Words.pkl','wb'))
    pickle.dump(self.vAR_classes,open('../Utility/DSAI_Tbot_Classes.pkl','wb'))

  def initialize_training(self):
    # initializing training data
    # prepare training data
    self.vAR_training = []
    vAR_output_empty = [0] * len(self.vAR_classes)

    for doc in self.vAR_documents:
        vAR_bag = []  # initializing bag of words
        vAR_pattern_words = doc[0]  # list of tokenized words for the pattern
        vAR_pattern_words = [self.vAR_lemmatizer.lemmatize(word.lower()) for word in vAR_pattern_words] # lemmatize each word

        # create our bag of words array with 1, if word match found in current pattern
        for w in self.vAR_words:
            vAR_bag.append(1) if w in vAR_pattern_words else vAR_bag.append(0)

        # output is a '0' for each tag and '1' for current tag (for each pattern)
        vAR_output_row = list(vAR_output_empty)
        vAR_output_row[self.vAR_classes.index(doc[1])] = 1

        self.vAR_training.append([vAR_bag, vAR_output_row])#bag with its class array
    
    random.shuffle(self.vAR_training)  # shuffle our features and turn into np.array
    self.vAR_training = np.array(self.vAR_training)

  def create_train_lists(self):
    # create train lists.
    # X - patterns, Y - intents
    self.vAR_train_x = list(self.vAR_training[:,0])
    self.vAR_train_y = list(self.vAR_training[:,1])

  def create_model(self):
    # Number of layers - 3 layers.
    # First layer 128 neurons
    # Second layer 64 neurons 
    # 3rd output layer contains vAR_train_y[0] number of neurons
    # equal to number of intents to predict output intent with softmax
    self.vAR_model = Sequential()
    self.vAR_model.add(Dense(128, input_shape=(len(self.vAR_train_x[0]),), activation='relu'))
    self.vAR_model.add(Dropout(0.5))
    self.vAR_model.add(Dense(64, activation='relu'))
    self.vAR_model.add(Dropout(0.5))
    self.vAR_model.add(Dense(len(self.vAR_train_y[0]), activation='softmax'))

  def compile_model(self):
    # Compile model.
    # Optimizer - Stochastic gradient descent (SGD)
    # Nesterov accelerated gradient gives good results for this model
    vAR_sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    self.vAR_model.compile(loss='categorical_crossentropy', optimizer = vAR_sgd, metrics=['accuracy'])

  def fit_and_save_model(self):
    #fitting the model
    # save the model in .h5 format
    vAR_hist = self.vAR_model.fit(np.array(self.vAR_train_x), np.array(self.vAR_train_y), epochs=200, batch_size=5)
    self.vAR_model.save('../Utility/DSAI_Tbot_Chatbot_Model.h5', vAR_hist)

    print("Model created")

  def RUN_MODEL(self):
    # method to run all methods
    self.store_object() # method call to store classes and words in pickle format
    self.initialize_training() # method call to prepare training data
    self.create_train_lists() #method call to prepare Train list of prepared training data
    self.create_model() #method call to create Sequential model of 3 layers
    # Compile the model with SGD Optimizer, Nesterov Accelerator using Categorical Crossentropy loss and accuracy Metric
    self.compile_model()
    self.fit_and_save_model() #method call to fit the model and save the trained model as DSAI_chatbot_model.h5

# Import Data
Input data from DSAI_intents.json is imported

In [3]:
vAR_data_file = open('../Utility/DSAI_Tbot_Intents.json').read()
vAR_intents = json.loads(vAR_data_file)

# Create Object
Creation of object for class ChatBot with the dataset

In [4]:
vAR_chat_bot_object = ChatBot(vAR_intents)

56 documents [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['Are', 'you', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Hay'], 'greeting'), (['See', 'you'], 'goodbye'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['catch', 'you', 'later'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['I', 'Appreciate', 'it'], 'thanks'), (['So', 'kind', 'of', 'you'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['I', 'really', 'appreciate', 'it', '!', '!', '!'], 'thanks'), (['how', 'you', 'doing', '?'], 'ask'), (['how', 'are', 'you', '?'], 'ask'), (['Hope', 'you', 'are', 'doing', 'good'], 'ask'), (['You', 'good', '?'], 'ask'), (['what', 'is', 'your', 'name'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['Who', 'are', 'You', '?'], 'name'), (['How', 'can', 'I', 'call', 'you', '?'], 'name'), (['Do', 'you', 'have', 'a', 'name', '?'], 'name'), (['Can', 'you', 'list', 'me', 'yo

# RUN MODEL

In [5]:
vAR_chat_bot_object.RUN_MODEL()

Epoch 1/200
12/12 [==============================] - 1s 6ms/step - loss: 2.7972 - accuracy: 0.0357
Epoch 2/200
12/12 [==============================] - 0s 6ms/step - loss: 2.6927 - accuracy: 0.0714
Epoch 3/200
12/12 [==============================] - 0s 5ms/step - loss: 2.5953 - accuracy: 0.1964
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 2.5950 - accuracy: 0.1429
Epoch 5/200
12/12 [==============================] - 0s 5ms/step - loss: 2.4325 - accuracy: 0.2679
Epoch 6/200
12/12 [==============================] - 0s 5ms/step - loss: 2.4313 - accuracy: 0.2679
Epoch 7/200
12/12 [==============================] - 0s 4ms/step - loss: 2.3311 - accuracy: 0.3036
Epoch 8/200
12/12 [==============================] - 0s 4ms/step - loss: 2.1337 - accuracy: 0.4286
Epoch 9/200
12/12 [==============================] - 0s 6ms/step - loss: 2.2088 - accuracy: 0.3750
Epoch 10/200
12/12 [==============================] - 0s 4ms/step - loss: 1.9452 - accuracy: 0.4464
Epoch 11/

12/12 [==============================] - 0s 3ms/step - loss: 0.1268 - accuracy: 0.9821
Epoch 84/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0972 - accuracy: 0.9643
Epoch 85/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0497 - accuracy: 1.0000
Epoch 86/200
12/12 [==============================] - 0s 2ms/step - loss: 0.2117 - accuracy: 0.9286
Epoch 87/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1949 - accuracy: 0.9107
Epoch 88/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1668 - accuracy: 0.9643
Epoch 89/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1469 - accuracy: 0.9643
Epoch 90/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1290 - accuracy: 0.9643
Epoch 91/200
12/12 [==============================] - 0s 2ms/step - loss: 0.1138 - accuracy: 0.9821
Epoch 92/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0895 - accuracy: 0.9643
Epoch 93/200


12/12 [==============================] - 0s 4ms/step - loss: 0.0575 - accuracy: 0.9821
Epoch 165/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 166/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0361 - accuracy: 0.9821
Epoch 167/200
12/12 [==============================] - 0s 4ms/step - loss: 0.0390 - accuracy: 1.0000
Epoch 168/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0455 - accuracy: 0.9821
Epoch 169/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9643
Epoch 170/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0405 - accuracy: 0.9821
Epoch 171/200
12/12 [==============================] - 0s 2ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 172/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0344 - accuracy: 0.9821
Epoch 173/200
12/12 [==============================] - 0s 3ms/step - loss: 0.0102 - accuracy: 1.0000
Epoc

# Copyright Notice:
Local and international copyright laws protect
this material. Repurposing or reproducing
this material without written approval from
DeepSphere.AI violates the law.

(c) DeepSphere.AI